# PyGMTSAR Python Notebook for example S1A_Stack_CPGF_T173

Tested on MacOS Catalina (Python 3.9) and Debian 10 (Python 3.7)

### I'm a freelancer and that's my free time Open Source project with GPL-3.0 License. If you find it useful you are able to sponsor my projects <a href="https://www.patreon.com/bePatron?u=54500608" data-patreon-widget-type="become-patron-button">Become a Patron!</a><script async src="https://c6.patreon.com/becomePatronButton.bundle.js"></script> or order additional research, development and support on <a href="https://www.upwork.com/freelancers/~01e65e8e7221758623">Upwork</a>

## @ Alexey Pechnikov, Sep, 2021, https://github.com/mobigroup

## Download and unpack the example, create processing directory

In [ ]:
import sys
import os

In [ ]:
import xarray as xr
import numpy as np
import dask
#from dask.diagnostics import ProgressBar
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')
import glob
from io import StringIO
import xmltodict

In [ ]:
# to show PDF
from wand.image import Image as WImage
# plotting modules
import hvplot.xarray  # noqa
import holoviews as hv
from holoviews import opts
from bokeh.models import FixedTicker
hv.extension('bokeh', 'matplotlib')
pd.options.plotting.backend = 'holoviews'

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# define common plot parameters
plot_opts = {'rasterize': True, 'xlabel':'Range', 'ylabel':'Azimuth', 'width':500, 'height':400}

## Custom modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from intf_ra2ll import intf_ra2ll_matrix, intf_ra2ll
from trans_ra2ll import trans_ra2ll_matrix
from PRM import PRM
from SBAS import SBAS

## Helper functions

In [ ]:
# stack by filepath for xr.open_mfdataset
def preprocess_dirname(ds):
    dates = ds.encoding["source"].split('/')[-1][:17].replace('_', ' ')
    #print (ds.encoding["source"], '->', stack)
    return ds.assign(date=dates)

def open_grids_stack(basedir, name):
    filenames = f'{basedir}/*{name}.grd'
    ds = xr.open_mfdataset(filenames, concat_dim='date', combine='nested',
                             preprocess=preprocess_dirname)['z']
    return ds

## Define parameters

In [ ]:
MASTER       = '2015-04-03'
WORKDIR      = 'raw'
DATADIR      = 'raw_orig'
DEMFILE      = 'topo/dem.grd'
#BASEDAYS     = 75
#BASEMETERS   = 75
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Init SBAS

In [ ]:
sbas = SBAS(DATADIR, DEMFILE, WORKDIR).set_master(MASTER)
sbas.to_dataframe()

In [ ]:
title = 'Sentinel1 Frame on DEM plus GCP'
#https://holoviz.org/tutorial/Composing_Plots.html
sbas.get_dem()[::4,::4].hvplot(invert=True, cmap='kbc', alpha=1, title=title) * \
    sbas.geoloc().plot.scatter(y='longitude', x='latitude', c='pixel', cmap='jet')

In [ ]:
title = 'Sentinel1 Frame DEM cropped using GCP'
#https://holoviz.org/tutorial/Composing_Plots.html
sbas.get_dem(geoloc=True)[::4,::4].hvplot(invert=True, cmap='kbc', alpha=1, title=title) * \
    sbas.geoloc().plot.scatter(y='longitude', x='latitude', c='pixel', cmap='jet')

## Stack Images (for a single subswath only)

In [ ]:
sbas.stack_parallel()

## SBAS Baseline

In [ ]:
baseline_pairs = sbas.baseline_pairs(days=100, meters=150)
baseline_pairs

## DEM in Radar Coordinates

In [ ]:
%%time

sbas.topo_ra()

In [ ]:
xr.open_dataarray(f'{WORKDIR}/topo_ra.grd').hvplot(cmap='kbc', title='Topo_ra', **plot_opts)

## Interferograms

In [ ]:
pairs = baseline_pairs[['ref_date', 'rep_date']]
pairs

In [ ]:
# we can just miss "func" argument when post-processing is not required
# define a postprocessing function for decimation, etc.
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').median()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing required)
sbas.intf_parallel(pairs, wavelength=400, func=decimator)

In [ ]:
phasefilts = open_grids_stack(WORKDIR, 'phasefilt')
phasefilts\
    .hvplot(by='date', width=320, height=280, subplots=True,
            xlabel='Range', ylabel='Azimuth',
            clim=(-np.pi,np.pi), cmap='gist_rainbow_r')\
    .cols(3).opts(title='Filtered Phase, [rad]')

In [ ]:
corrs = open_grids_stack(WORKDIR, 'corr')
corrs\
    .hvplot(by='date', width=320, height=280, subplots=True,
            xlabel='Range', ylabel='Azimuth',
            clim=(0, 0.8), cmap='gray')\
    .cols(3).opts(title='Correlation')

## Unwrapping

In [ ]:
pairs

In [ ]:
# generate a custom snaphu config file and use it as argument "conf" value
# conf = self.PRM().snaphu_config(defomax=0)

# we can just miss "func" argument when post-processing is not required
# define a post-processing function to crop and interpolate low-coherence areas, etc.
#cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)
cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

# default parameters: threshold=0.1, conf=None, func=None (no postprocessing required)
sbas.unwrap_parallel(pairs, threshold=CORRLIMIT, func=cleaner)

In [ ]:
unwraps = open_grids_stack(WORKDIR, 'unwrap')
unwraps\
    .hvplot(by='date', width=320, height=280, subplots=True, cmap='jet',
            xlabel='Range', ylabel='Azimuth',
            clim=tuple(np.nanquantile(unwraps, [0.01, 0.99])))\
    .cols(3).opts(title='Unwrapped Phase, [rad]')

## Bonus: Inverted Interferograms

We are able to build reverse-ordered interferograms by a simple hand move

In [ ]:
pairs_inverted = baseline_pairs[['rep_date', 'ref_date']]
pairs_inverted

In [ ]:
#sbas.intf_parallel(pairs_inverted, wavelength=400, func=decimator)